# Transformação Camada Silver

In [0]:
#Leitura
bronze_path = "/Volumes/workspace/default/bronze_accident"
df_bronze = spark.read.format("delta").load(bronze_path)
display(df_bronze)

In [0]:
df_bronze.printSchema()

In [0]:
#Padronização
from pyspark.sql.functions import col

df_silver = df_bronze
for c in df_silver.columns:
    df_silver = df_silver.withColumnRenamed(
        c,
        c.lower()
         .replace(" ", "_")
         .replace("ç", "c")
         .replace("ã", "a")
         .replace("õ", "o")
         .replace("á", "a")
         .replace("é", "e")
         .replace("í", "i")
         .replace("ó", "o")
         .replace("ú", "u")
    )

In [0]:
from pyspark.sql.functions import year, month, dayofmonth, col

df_silver = (
    df_silver
    .withColumn("ano", year(col("data_inversa")))
    .withColumn("mes", month(col("data_inversa")))
    .withColumn("dia", dayofmonth(col("data_inversa")))
)

In [0]:
#nulos
df_silver = (
    df_silver
    .fillna({
        "mortos": 0,
        "feridos": 0
    })
)

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.default.silver_accident;

In [0]:
silver_path = "/Volumes/workspace/default/silver_accident"

(
    df_silver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silver_path)
)

In [0]:
df_silver_check = spark.read.format("delta").load(silver_path)
print("Bronze rows:", df_bronze.count())
print("Silver rows:", df_silver_check.count())

In [0]:
display(df_silver.limit(10))

In [0]:
df_silver.printSchema()